In [1]:
import cv2 as cv
import numpy as np
import torch
import matplotlib.pyplot as plt
import mediapipe as mp
import uuid  # unique identifier
import os
import time
import math

objc[93607]: Class CaptureDelegate is implemented in both /Users/adilbhatti/anaconda3/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x1696124d0) and /Users/adilbhatti/anaconda3/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x147d88860). One of the two will be used. Which one is undefined.
objc[93607]: Class CVWindow is implemented in both /Users/adilbhatti/anaconda3/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x169612520) and /Users/adilbhatti/anaconda3/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x1223aca68). One of the two will be used. Which one is undefined.
objc[93607]: Class CVView is implemented in both /Users/adilbhatti/anaconda3/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x169612548) and /Users/adilbhatti/anaconda3/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x1223aca90). One of the two will be used. Which one is undefined.
objc[93607]: Class CVSlider is implemented in both /U

In [2]:
# Model
midas = torch.hub.load('intel-isl/MiDaS', 'MiDaS_small')
midas.to('cpu')
midas.eval()

# Transforms
transforms = torch.hub.load('intel-isl/MiDaS', 'transforms')
transform = transforms.small_transform

Using cache found in /Users/adilbhatti/.cache/torch/hub/intel-isl_MiDaS_master


Loading weights:  None


Using cache found in /Users/adilbhatti/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master
Using cache found in /Users/adilbhatti/.cache/torch/hub/intel-isl_MiDaS_master


In [2]:
# hands class
mp_hands = mp.solutions.hands

# setting up hand functions
hands = mp_hands.Hands(min_detection_confidence = 0.70)

# drawing class
mp_draw = mp.solutions.drawing_utils

In [3]:
# Returns resulting image and results
def detectHandLandmarks(img, hands, draw=True):
    output_img = img.copy()
    imgRGB = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    
    if results.multi_hand_landmarks and draw:
        for hand_landmark in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(image = output_img, landmark_list = hand_landmark,
                                      connections = mp_hands.HAND_CONNECTIONS,
                                      landmark_drawing_spec = mp_draw.DrawingSpec(color=(0,0,255),
                                                                                  thickness=5, circle_radius=5),
                                      connection_drawing_spec=mp_draw.DrawingSpec(color = (0,0,0),
                                                                                     thickness = 2, circle_radius = 10))
    return (output_img, results)      

# Text
font                   = cv.FONT_HERSHEY_SIMPLEX
fontScale              = 2
fontColor              = (255,255,0)
thickness              = 5
lineType               = 2

In [4]:
rev_keys =     [['`', '1', '2', '3','4','5','6','7','8','9','0','-','=','delete'],
            ['tab', 'q','w','e','r','t','y','u','i','o','p','[',']','\\'],
            ['capslock','a','s','d','f','g','h','j','k','l',';',"'", 'return'],
            ['shift', 'z','x','c','v','b','n','m',',','.','/','shift','shift']]

keys = [layer[::-1] for layer in rev_keys]

positions = []
for j in range(4):
    positions.append([])
    for i in range(14 if j < 2 else 13):
        px, py = 130 * i + 70 + 80*j,  290 + 165 * j
        if j > 1:
            px -= 45
            py += 10
        positions[-1].append((px,py))

from itertools import chain
flat_keys = list(chain.from_iterable(keys))
flat_rev = list(chain.from_iterable(rev_keys))
flat_pos = list(chain.from_iterable(positions))
layout = {pos: key for pos, key in zip(flat_pos, flat_keys)}

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [6]:
cap = cv.VideoCapture(0)
ptime = ctime = 0
blank = np.zeros((1080, 1920, 3), np.uint8)

dim = (1920, 1080)
keyboard = cv.resize(cv.imread('Apple_Magic_Keyboard_-_US_remix_transparent.png'), dim, interpolation=cv.INTER_AREA)
while cap.isOpened():
    keyboard = cv.resize(cv.imread('Apple_Magic_Keyboard_-_US_remix_transparent.png'), dim, interpolation=cv.INTER_AREA)
    hando = np.zeros((1080, 1920, 3), np.uint8)
    ret, img = cap.read()
    if not ctime:
        h, w, c = img.shape
    if ret:
        rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        #imgbatch = transform(rgb).to('cpu')
        
#         with torch.inference_mode():
#             pred = midas(imgbatch)
#             pred = torch.nn.functional.interpolate(
#                 pred.unsqueeze(1),
#                 size=rgb.shape[:2],
#                 mode = 'bicubic',
#                 align_corners=False
#             ).squeeze()
#             dists = pred.cpu().numpy()
#             comps = dists.copy()
        
        drawn_img, results = detectHandLandmarks(img, hands)
        ctime = time.time()
        fps = 1/(ctime-ptime)
        ptime = ctime
        result = hands.process(rgb)
        cv.putText(img,f'FPS: {int(fps)}', (50, 70), font, fontScale,fontColor,thickness,lineType)
        # Labeling the landmarks also calculates if thumbs is highest y coordinate
        key = ''
        if results.multi_hand_landmarks:
            for handLms in results.multi_hand_landmarks:
                qx, qy = handLms.landmark[6].x * w, handLms.landmark[6].y * h
                px, py = handLms.landmark[8].x * w, handLms.landmark[8].y * h
                wrist = (handLms.landmark[0].x * w, handLms.landmark[0].y * h)
                anchor_point = (handLms.landmark[9].x * w, handLms.landmark[9].y * h)
                basis = math.dist(anchor_point, wrist)
                if(math.dist((0, py), (0, qy)) < 2*basis/5):  
                    for pos, let in layout.items():
                        if pos in positions[1] or pos in positions[2]:
                            if math.dist(pos, (px+50, py)) < 50:
                                key = let
                                break
                        elif pos in positions[0]:
                            if math.dist(pos, (px-50, py)) < 50:
                                key = let
                                break
                        else:
                            if math.dist(pos, (px+150, py)) < 50:
                                key = let
                                break
                
                #cv.circle(blank,(int(px),int(py)), 20, (0,0,255), -1)
                mp_draw.draw_landmarks(image = img, landmark_list = handLms,
                                          connections = mp_hands.HAND_CONNECTIONS,
                                          landmark_drawing_spec = mp_draw.DrawingSpec(color=(0,0,255),
                                                                                      thickness=5, circle_radius=5),
                                          connection_drawing_spec=mp_draw.DrawingSpec(color = (255,255,255),
                                                                                         thickness = 10, circle_radius = 10))
                mp_draw.draw_landmarks(image = hando, landmark_list = handLms,
                                          connections = mp_hands.HAND_CONNECTIONS,
                                          landmark_drawing_spec = mp_draw.DrawingSpec(color=(0,0,255),
                                                                                      thickness=5, circle_radius=5),
                                          connection_drawing_spec=mp_draw.DrawingSpec(color = (255,0,0),
                                                                                         thickness = 10, circle_radius = 20))
        else:
            blank = np.zeros((1080, 1920, 3), np.uint8)
        #dists *= (255.0/dists.max())
        #cv.imshow('Dist', np.uint8(dists))
        
        hando = cv.flip(hando, 1)
        edges = cv.Canny(keyboard, 100, 200)
        over = cv.addWeighted(img,0.8,blank,1,0)
        keylap = cv.addWeighted(img, 1, keyboard, 0.2,0)
        keyboard = cv.addWeighted(keyboard, 0.2, hando, 1, 0)
        cv.putText(keyboard,f'CLICK!{key}', (w - 350, 70), font, fontScale,fontColor,thickness,lineType)
        for spot, let in zip(flat_pos, flat_rev):
            cv.putText(keyboard,let, spot, font//2, fontScale,fontColor,thickness,lineType)
            cv.circle(keyboard, spot, 50, (0,255,0), 1)
        cv.imshow('Drawing', cv.flip(keylap, 1))
        cv.imshow('Blank', cv.flip(over, 1))
        cv.imshow('KEY', keyboard)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv.destroyAllWindows()

In [41]:

while True:
    cv.imshow("image", img)

KeyboardInterrupt: 